In [1]:
##modules
#%matplotlib widget
#%matplotlib inline
#
#%matplotlib qt
import mne
import numpy as np
import matplotlib
# Establecer un backend interactivo, como 'Qt5Agg', 'GTK3Agg', etc.
# Esto depende de los backends disponibles en tu sistema.

import matplotlib.pyplot as plt

matplotlib.use('TkAgg')  # Asegúrate de que este backend está instalado.

import pandas as pd 
import os
import sys

from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

from os.path import join as pathjoin
from time import time

from pathlib import Path

from autoreject import AutoReject

In [2]:
# Variables path
layer_script = "block"

#process=""
disco="g"

# subj = "sub-A2004"

#subj = sys.argv[1] ## name of participant list

# Carpeta general
datadir = Path(f"{disco}:\\PROYECTO_SELF")

# #carpetas generales de datos
brain_vision_dir = datadir /"Data_self"/"Export_acw_Brain_Vision"
fif_data = datadir /"Data_self"/ "fif_data" 


#carpeta analysis
output_analysis= datadir / "output_analysis"

acw_analysis_path = output_analysis / "acw"
acw_analysis_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de preprocesado
# output_preproc = datadir / "output_preproc"

# preproc_path = output_preproc / f"preproc_{layer_script}"
# preproc_path.mkdir(parents=True, exist_ok=True)
 
# # Carpeta de epocas "sucias"
# epochs_path = preproc_path / f"epochs_{layer_script}"
# epochs_path.mkdir(parents=True, exist_ok=True) 

# # Carpeta de ICA
# ICA_path = preproc_path / f"ICA_{layer_script}"
# ICA_path.mkdir(parents=True, exist_ok=True)

# # Épocas limpias
# epochs_clean_path = preproc_path / f"epochs_clean_{layer_script}"
# epochs_clean_path.mkdir(parents=True, exist_ok=True)

# #epocas evoked
# evoked_path = Path(preproc_path) / f"evoked_{layer_script}"
# evoked_path.mkdir(parents=True, exist_ok=True)
 

# # Definir la carpeta de output_source antes de usarla
# output_source = Path(r"g:\MOUS_204\output_source")

# source_path = output_source / f"source_{layer_script}"
# source_path.mkdir(parents=True, exist_ok=True)

# #raw_hsp es el raw con fiducials cargados
# raw_hsp_path = source_path / f"raw_hsp"
# raw_hsp_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de forward solution
# fwd_path = source_path / f"fwd"
# fwd_path.mkdir(parents=True, exist_ok=True)

# # Carpeta de inverse solution
# inverse_path = source_path / f"inverse"
# inverse_path.mkdir(parents=True, exist_ok=True)


# mne.utils.set_config('SUBJECTS_DIR', r'\\wsl$\Ubuntu-20.04\usr\local\freesurfer\subjects', set_env=True)

In [5]:
import re
from pathlib import Path

# Definir la carpeta donde buscar los archivos
brain_vision_dir = Path(r'G:\PROYECTO_SELF\Data_self\Export_acw_Brain_Vision')

# Definir combinaciones posibles (Sx - Sy)
combinaciones = ["S1 - S4","S1 - S5", "S2 - S4", "S2 - S5", "S3 - S4", "S3 - S5"]

# Crear un diccionario para almacenar los patrones
patrones = {}

# Generar patrones para S01 hasta S90
for sujeto in range(1, 91):  # 01 a 90
    sujeto_str = f"S{sujeto:02d}"  # Asegura formato S01, S02 ... S90
    for combinacion in combinaciones:
        clave = f"{sujeto_str}_{combinacion}"  # Ejemplo: S01_S1 - S5
        patron = re.compile(rf"^{sujeto_str}_vis\d*_{combinacion}\.vhdr$")
        patrones[clave] = patron

# Obtener todos los archivos .vhdr en la carpeta
archivos = list(brain_vision_dir.glob("*.vhdr"))

# Filtrar archivos que coincidan con los patrones generados
archivos_filtrados = {nombre: [] for nombre in patrones.keys()}

for archivo in archivos:
    for nombre, patron in patrones.items():
        if patron.match(archivo.name):
            archivos_filtrados[nombre].append(archivo.name)

# Agrupar archivos por sujeto (S01, S02, ...)
archivos_por_sujeto = {}

for nombre, archivos in archivos_filtrados.items():
    if archivos:  # Solo muestra si hay archivos encontrados
        sujeto = nombre.split("_")[0]  # Extraer el sujeto (Ej: "S01")
        if sujeto not in archivos_por_sujeto:
            archivos_por_sujeto[sujeto] = []  # Crear lista para el sujeto
        archivos_por_sujeto[sujeto].extend(archivos)  # Agregar archivos



for sujeto, archivos in archivos_por_sujeto.items():
    print(f"\nProcesando sujeto {sujeto}:")

    epocas_sujeto = {}  # Diccionario para almacenar las épocas por condición

    for archivo in archivos:
        file = archivo[:-5]  # Elimina los últimos 5 caracteres (".vhdr")
        print(f"  - {file}")  # Muestra el nombre sin .vhdr
        
        # Leer el archivo BrainVision
        data = brain_vision_dir / (file + ".vhdr")
        raw = mne.io.read_raw_brainvision(data, preload=True)

        # Establecer M1 y M2 como referencia común
        raw.set_eeg_reference(ref_channels=['M1', 'M2'])

        # Marcar los canales oculares como EOG
        ocular_channels = ['HEOG+', 'HEOG', 'VEOG+', 'VEOG']
        raw.set_channel_types({ch: 'eog' for ch in ocular_channels})

        # Marcar el canal GSR como tipo GSR
        raw.set_channel_types({'GSR': 'misc'})  # GSR suele marcarse como 'misc' en MNE

        # Extraer la condición (Ejemplo: "S1 - S4", "S2 - S5", etc.)
        condicion = file.split("_")[-1]  # Toma la última parte después de "_"

        # Crear epochs de 9.2s
        epochs = mne.make_fixed_length_epochs(
            raw,duration=9.2, reject_by_annotation=True, preload=True
        )

        subj_fif = fif_data / f"{sujeto}_epochs"
        subj_fif.mkdir(parents=True, exist_ok=True)

        epochs.save(subj_fif /f"{sujeto}_{condicion}-epo.fif", overwrite=True)


Procesando sujeto S01:
  - S01_vis2_S1 - S4
Extracting parameters from G:\PROYECTO_SELF\Data_self\Export_acw_Brain_Vision\S01_vis2_S1 - S4.vhdr...
Setting channel info structure...
Reading 0 ... 87399  =      0.000 ...   174.798 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
19 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 19 events and 4600 original time points ...
0 bad epochs dropped


C:\Users\UCM\AppData\Local\Temp\ipykernel_8624\362719528.py:55: RuntimeWarning: No coordinate information found for channels ['HEOG+', 'HEOG', 'VEOG+', 'VEOG', 'M1', 'M2']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(data, preload=True)
C:\Users\UCM\AppData\Local\Temp\ipykernel_8624\362719528.py:55: RuntimeWarning: Not setting positions of 7 misc channels found in montage:
['HEOG+', 'HEOG', 'VEOG+', 'VEOG', 'M1', 'GSR', 'M2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(data, preload=True)
C:\Users\UCM\AppData\Local\Temp\ipykernel_8624\362719528.py:62: RuntimeWarning: The unit for channel(s) HEOG, HEOG+, VEOG, VEOG+ has changed from NA to V.
  raw.set_channel_types({ch: 'eog' for ch in ocular_channels})


  - S01_vis2_S1 - S5
Extracting parameters from G:\PROYECTO_SELF\Data_self\Export_acw_Brain_Vision\S01_vis2_S1 - S5.vhdr...
Setting channel info structure...
Reading 0 ... 96599  =      0.000 ...   193.198 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 21 events and 4600 original time points ...
0 bad epochs dropped


C:\Users\UCM\AppData\Local\Temp\ipykernel_8624\362719528.py:55: RuntimeWarning: No coordinate information found for channels ['HEOG+', 'HEOG', 'VEOG+', 'VEOG', 'M1', 'M2']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(data, preload=True)
C:\Users\UCM\AppData\Local\Temp\ipykernel_8624\362719528.py:55: RuntimeWarning: Not setting positions of 7 misc channels found in montage:
['HEOG+', 'HEOG', 'VEOG+', 'VEOG', 'M1', 'GSR', 'M2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(data, preload=True)
C:\Users\UCM\AppData\Local\Temp\ipykernel_8624\362719528.py:62: RuntimeWarning: The unit for channel(s) HEOG, HEOG+, VEOG, VEOG+ has changed from NA to V.
  raw.set_channel_types({ch: 'eog' for ch in ocular_channels})


  - S01_vis2_S2 - S4
Extracting parameters from G:\PROYECTO_SELF\Data_self\Export_acw_Brain_Vision\S01_vis2_S2 - S4.vhdr...
Setting channel info structure...
Reading 0 ... 78199  =      0.000 ...   156.398 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
17 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 17 events and 4600 original time points ...
0 bad epochs dropped


...

0 bad epochs dropped
  - S06_vis_S3 - S4
Extracting parameters from G:\PROYECTO_SELF\Data_self\Export_acw_Brain_Vision\S06_vis_S3 - S4.vhdr...
Setting channel info structure...
Reading 0 ... 105799  =      0.000 ...   211.598 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
23 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 23 events and 4600 original time points ...


C:\Users\UCM\AppData\Local\Temp\ipykernel_8624\362719528.py:55: RuntimeWarning: No coordinate information found for channels ['HEOG+', 'HEOG', 'VEOG+', 'VEOG', 'M1', 'M2']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(data, preload=True)
C:\Users\UCM\AppData\Local\Temp\ipykernel_8624\362719528.py:55: RuntimeWarning: Not setting positions of 7 misc channels found in montage:
['HEOG+', 'HEOG', 'VEOG+', 'VEOG', 'M1', 'GSR', 'M2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(data, preload=True)
C:\Users\UCM\AppData\Local\Temp\ipykernel_8624\362719528.py:62: RuntimeWarning: The unit for channel(s) HEOG, HEOG+, VEOG, VEOG+ has changed from NA to V.
  raw.set_channel_types({ch: 'eog' for ch in ocular_channels})


0 bad epochs dropped
  - S06_vis_S3 - S5
Extracting parameters from G:\PROYECTO_SELF\Data_self\Export_acw_Brain_Vision\S06_vis_S3 - S5.vhdr...
Setting channel info structure...
Reading 0 ... 110399  =      0.000 ...   220.798 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
24 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 24 events and 4600 original time points ...


C:\Users\UCM\AppData\Local\Temp\ipykernel_8624\362719528.py:55: RuntimeWarning: No coordinate information found for channels ['HEOG+', 'HEOG', 'VEOG+', 'VEOG', 'M1', 'M2']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(data, preload=True)
C:\Users\UCM\AppData\Local\Temp\ipykernel_8624\362719528.py:55: RuntimeWarning: Not setting positions of 7 misc channels found in montage:
['HEOG+', 'HEOG', 'VEOG+', 'VEOG', 'M1', 'GSR', 'M2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(data, preload=True)
C:\Users\UCM\AppData\Local\Temp\ipykernel_8624\362719528.py:62: RuntimeWarning: The unit for channel(s) HEOG, HEOG+, VEOG, VEOG+ has changed from NA to V.
  raw.set_channel_types({ch: 'eog' for ch in ocular_channels})


0 bad epochs dropped
